In [71]:
# --- 2020-01 CAU
# --- Natural Language Processing and Information Retrieval
# --- Sentiment Analysis Team Project
# --- 
# --- 

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs
import xlsxwriter
import time

# webdriver dir
browser = webdriver.Chrome('./chromedriver.exe') # watch out the version

In [104]:
# --- Saving results as .xlsx file
def savexlsx(xlsxfilename, array):
    with xlsxwriter.Workbook(xlsxfilename) as workbook:
        worksheet = workbook.add_worksheet()

        # Add a bold format to use to highlight cells
        bold = workbook.add_format({'bold': True})

        for row_num, data in enumerate(array):
            if row_num == 0:
                worksheet.write('A1','title',bold)
                worksheet.write('B1','artist',bold)
                worksheet.write('C1','lyric',bold)
                continue
            worksheet.write_row(row_num, 0, data)
    workbook.close()

for year in range(2007, 2019):
    print("\n=====", year)
    chart_url = "https://www.billboard.com/charts/year-end/"+str(year)+"/hot-100-songs"
    browser.get(chart_url)
    
    # ----- collecting top 100 songs over years ----- #
    chart = []
    for rank in range(1, 101):
        title = browser.find_element_by_xpath("(//div[@class='ye-chart-item__title'])["+str(rank)+"]").text
        artist = browser.find_element_by_xpath("(//div[@class='ye-chart-item__artist'])["+str(rank)+"]").text
        #print("[{}] {} - {}".format(rank, title, artist))
        chart.append([title, artist, ''])
    # ----------------------------------------------- #
    
    
    # --------- Search lyrics for 100 songs --------- #
    # Be careful! You could be BLOCKED by the website #
    for rank in range(0, 100):
        org_title = chart[rank][0].replace('/', ' ')         # error_handling (ex.cupid's chokehead/breakfast in america)
        org_artist = chart[rank][1].split(' Featuring',1)[0] # remove featuring artists _error handling
        org_artist = org_artist.split(' (Featuring',1)[0]
        print(rank, org_title, org_artist)
        browser.get("https://www.musixmatch.com/search/"+org_title+' '+org_artist)
        time.sleep(2)
        
        # Need to check which searched result is matched to original info over webpage but we passed this process
        # because [Best results] song is nearly perfectly same
        """
        title = browser.find_element_by_xpath("(//h2[@class='media-card-title'])[1]")
        artist = browser.find_element_by_xpath("(//h3[@class='media-card-subtitle'])[1]").text.split('feat',1)[0]
        #print(org_title,'-',org_artist, '/', title.text,'-',artist)
        if title.text == org_title and artist == org_artist:
            title.click()
        """
        try:
            browser.find_element_by_xpath("//a[@class='title']").click()
            time.sleep(1)

            # Sometimes, there're some lyrics which are judged as errored or warning by the website
            # But we ignored this notice and collect them no matter what state it is
            try:
                lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__ok'])[1]").text
                # if the content is divided, concatenate them
                try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__ok'])[2]").text
                except: continue
            except:
                try:
                    lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__error'])[1]").text
                    try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__error'])[2]").text
                    except: continue
                except:
                    try:
                        lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__warning'])[1]").text
                        try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__warning'])[2]").text
                        except:continue
                    except: continue # Lyrics are not available (ex. restricted)
            #print(lyric)
            chart[rank][2] = lyric
        except NoSuchElementException: # No searching results
            print("!")
    # ----------------------------------------------- #
    #print(chart)
    savexlsx('./'+str(year)+'_data.xlsx', chart)


===== 2007
0 Irreplaceable Beyonce
1 Umbrella Rihanna
2 The Sweet Escape Gwen Stefani
3 Big Girls Don't Cry Fergie
4 Buy U A Drank (Shawty Snappin') T-Pain
5 Before He Cheats Carrie Underwood
6 Hey There Delilah Plain White T's
7 I Wanna Love You Akon
8 Say It Right Nelly Furtado
9 Glamorous Fergie
10 Don't Matter Akon
11 Girlfriend Avril Lavigne
12 Makes Me Wonder Maroon 5
13 Party Like A Rockstar Shop Boyz
14 Smack That Akon
15 This Is Why I'm Hot Mims
16 It's Not Over Daughtry
17 The Way I Are Timbaland
18 Fergalicious Fergie
19 Crank That (Soulja Boy) Soulja Boy Tell'em
20 Give It To Me Timbaland
21 What Goes Around...Comes Around Justin Timberlake
22 Cupid's Chokehold Breakfast In America Gym Class Heroes
23 How To Save A Life The Fray
24 Home Daughtry
25 My Love Justin Timberlake
26 Stronger Kanye West
27 We Fly High Jim Jones
28 U + Ur Hand P!nk
29 Walk It Out Unk
30 Beautiful Girls Sean Kingston
31 This Ain't A Scene, It's An Arms Race Fall Out Boy
32 Bartender T-Pain
33 Pop, 

90 I Do Not Hook Up Kelly Clarkson
91 Green Light John Legend
92 People Are Crazy Billy Currington
93 Whatever It Is Zac Brown Band
94 Already Gone Kelly Clarkson
95 Goodbye Kristinia DeBarge
96 Say Hey (I Love You) Michael Franti & Spearhead
97 Pop Champagne Jim Jones & Ron Browz
98 Pretty Wings Maxwell
99 Never Say Never The Fray

===== 2010
0 TiK ToK Ke$ha
1 Need You Now Lady Antebellum
2 Hey, Soul Sister Train
3 California Gurls Katy Perry
4 OMG Usher
5 Airplanes B.o.B
6 Love The Way You Lie Eminem
7 Bad Romance Lady Gaga
8 Dynamite Taio Cruz
9 Break Your Heart Taio Cruz
10 Nothin' On You B.o.B
11 I Like It Enrique Iglesias
12 BedRock Young Money
13 In My Head Jason Derulo
14 Rude Boy Rihanna
15 Telephone Lady Gaga
16 Teenage Dream Katy Perry
17 Just The Way You Are Bruno Mars
18 Cooler Than Me Mike Posner
19 Imma Be The Black Eyed Peas
20 Empire State Of Mind Jay-Z + Alicia Keys
21 DJ Got Us Fallin' In Love Usher
22 Billionaire Travie McCoy
23 Not Afraid Eminem
24 Replay Iyaz
25 S

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"(//div[@class='ye-chart-item__title'])[100]"}
  (Session info: chrome=83.0.4103.61)
